# Machine learning (60 minutes)
ACPSEM Machine Learning Workshop 2019, 29 - 31 May 2019

Yu Sun, yu.sun@sydney.edu.au

University of Sydney

This session demonstrates several key concepts in machine learning:
* Correlation
* Data partitioning
* Model fitting
* Performance evaluation (including sensititvy and specificity)

## Correlation (15 minutes)
The part shows how to compute the correlation coefficient of two vectors. First, we generate two *random* vectors.

In [0]:
# Generate two random vectors
import numpy as np
np.random.seed(1234)           # For reproducibility purposes
x1 = np.random.randn(1000)
x2 = np.random.randn(1000)
print('x1 first 50 items:', x1[:50])
print('x2 first 50 items:', x2[:50])

In [0]:
# Visualise
import matplotlib.pyplot as plt
plt.scatter(x=x1, y=x2, alpha=0.5)
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Scatter plot of x1 and x2')
plt.show()

In [0]:
# Compute the Pearson correlation
import scipy.stats
print('Pearson correlation: %.3f, likelihood of non-correlated systems: %.3f' \
      % scipy.stats.pearsonr(x1, x2))
print('Spearman correlation: %.3f, likelihood of non-correlated systems: %.3f' \
      % scipy.stats.spearmanr(x1, x2))


Recall that these two vectors are independent, so that the correlation is almost zero. What about some correlated data?

In [0]:
# Gnerate some non-independent data
np.random.seed(0)
x3 = np.random.randn(1000)
noise = np.random.randn(1000)
x4 = np.exp(x3) + noise

In [0]:
# Visualise
import matplotlib.pyplot as plt
plt.scatter(x=x3, y=x4, alpha=0.5)
plt.xlabel('x3')
plt.ylabel('x4')
plt.title('Scatter plot of x3 and x4')
plt.show()

Compute the Pearson and Spearman correlation coefficients for `x3` and `x4`.

In [0]:
# You code goes here:





## Classifier training (45 minutes)
In this part, we will create a learning task and go through the key steps.

### Geneate data for classification (5 minutes)

In [0]:
# Import the scikit-learn library
import sklearn
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split

In [0]:
# Generate some dummy data
X, y = make_classification(n_samples=300, n_features=2, n_redundant=0, 
                           n_informative=2, random_state=10, 
                           n_clusters_per_class=1)

In [0]:
# Visualise the data
# First we define a function to do the scatter plot
def plotData(x, y, title='Title'):
  'Scatter plot of the x, colour indicated by y'
  import matplotlib.pyplot as plt
  import matplotlib.patches as mpatches
  plt.style.use('ggplot')
  plt.scatter(x[:,0], x[:, 1], 
              c=y, 
              cmap=ListedColormap(['orange', 'darkcyan']), # for 0 and 1
              edgecolors='k',
              s=80)
  plt.xlabel('x1')
  plt.ylabel('x2')
  plt.title(title)
  patches = (mpatches.Patch(color='orange', label='Negative'),
             mpatches.Patch(color='darkcyan', label='Positive'))
  plt.legend(handles=patches)
  plt.show()

In [0]:
plotData(X, y, 'Generated data for classification')

### Data partitioning  (5 minutes)

In [0]:
# Split the data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [0]:
# Check the dimensions
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Exercise: plot the training data and test data.

### Model fitting  (5 minutes)

In [0]:
# Train a support vector machine (SVM) model
from sklearn.svm import SVC
svm_model = SVC(gamma=1, C=1)

In [0]:
# Train the model
svm_model.fit(X_train, y_train)

### Performance evalution  (30 minutes)

#### Apply the model on the test data

In [0]:
# Predict on the test data
svm_pred = svm_model.predict(X_test)
print('Prediction (SVM): ', svm_pred)
print('Ground truth: ', y_test)

We need to choose a proper metric to *quantitatively* measure the performance of the model. The most straightfoward metric is **accuracy**.

#### Accuracy
Defined as the percentage that the prediction agrees with the ground truth (`y_test`).

In [0]:
# Compute the accuracy for the SVM model
sum(svm_pred == y_test) / svm_pred.size

Explore the different component in the previous expression for accuracy, for example: what does it mean for `svm_pred == y_test`? Use `print()` to inspect each component.

In [0]:
# You code goes here:
# e.g.
print(svm_pred==y_test)





Exercise: for a learning task of imbalanced data (say 90% positive, 10% negative). If a model simply predict everything as positive, what's the accuracy? In another word, is accuracy suitable for all learning scenarios?

The answer to that leads to sensitivity and specificity analysis.

#### Sensitivity and Specificity
Write a function `calSS()` to calculate sensitivity and specificity given their definitions: $$Sensitivity = TP / (TP+FN)$$ $$Specificity = TN / (TN+FP)$$

$TP$: true positive;
$TN$: true negative;
$FP$: false positive;
$FN$: false negative.

In [0]:
def calSS(pred, truth):
    '''
    Compute the sensitivity and specificity of a prediction given the ground truth
    Input:
        pred - predicted outcome (binary)
        truth - ground truth (binary)
    Return: 
        A tuple of sensitivity and specificity value
    '''
    tp = np.logical_and(pred==1, truth==1) # True Positivee (TP)
    fp = np.logical_and(pred==1, truth==0) # False Positive (FP)
    tn = np.logical_and(pred==0, truth==0) # True Negative (TN)
    fn = np.logical_and(pred==0, truth==1) # False Negative (FN)

    sen = tp.sum() / (tp.sum() + fn.sum())
    spe = tn.sum() / (tn.sum() + fp.sum())
    return sen, spe

In [0]:
# Calculate the results on our previous model
print("Sensitivity: %.2f, specificity: %.2f" % calSS(svm_pred, y_test))

Back to the last question, in an imbalanced dataset (90% positive, 10% negative), if a model simply output everything as positive, it will have a high sensitivity. However, it won't have a high specificity. 

*In another word, sensitivity and specifity give you a 'stereo' overview of the model.*

We will use (sen, spe) as a short hand notation for a sensitivity and specificity pair.

#### Receiver operating characteristics (ROC) analysis

Some models (e.g. the logistic regression) output a probabilistic output rather than a binary output. Now we train a logistic regression using the same training data.

In [0]:
# Initiate the logistic regression model
lor_model = sklearn.linear_model.LogisticRegression(solver='lbfgs')
# Train the model
lor_model.fit(X_train, y_train)
# Test the model
lor_pred = lor_model.predict_proba(X_test)
print(lor_pred)

For each row in the output, the two numbers give the probability that each test sample falls into the two categories. Notice, they sum up to 1.0 for each row.

An issue arises as we cannot directly compute the (sen, spe) now. But what we can do is the first threshold the value, i.e. turn each row to a binary decision, then compute the (sen, spe). 

We will define a function `thres()` to do that.

In [0]:
def thres(pred, t):
    '''
    Convert a continuous output to binary output using a threshold.
    Input:
        pred - the prediction vector
        t - a threshold
    Output:
        A binary vector
    '''
    return (pred >= t) * 1

In [0]:
# Test it on the logistic regression output using a threshold of 0.5
lor_pred_binary = thres(lor_pred[:, 1], t=0.5)
print(lor_pred_binary)

In [0]:
# Now we can compute the (sen, spe)
print("Sensitivity: %.2f, specificity: %.2f" % calSS(lor_pred_binary, y_test))

The result above is dependent on the *threshold*. For a given threshold, we can convert the continuous output into binary. Then we can compute the sensitivity and specificity.

What if we create **a series of thresholds** and compute **the corresponding (sen, spe)** respectively?

In [0]:
# Create a series of thresholds
ts = range(0, 11)

# Create variables to hold the (sen, spe) pairs
lor_pred_sen = []
lor_pred_spe = []

# Iterate through each threshold and compute the corresponding (sen, spe) pairs
for eachT in ts:
    sen, spe = calSS(thres(lor_pred[:, 1], t=eachT/10.0), y_test)
    lor_pred_sen.append(sen)
    lor_pred_spe.append(spe)
    
# Print the result
for i in range(0, len(ts)):
    print('Thresholded at %s, sensitivity: %.3f, specificity: %.3f' % 
          (i/10.0, lor_pred_sen[i], lor_pred_spe[i]))

Plot the (sen, spe) points, what do we get? This will give the ROC curve.

In [0]:
# Plot the (sen, spe) pairs
import numpy as np
plt.step(x=np.array(lor_pred_spe), y=np.array(lor_pred_sen), where='post')
plt.xlabel('Specificity')
plt.ylabel('Sensitivity')
plt.title('Receiver Operating Charcteristics Curve')

By convention, the x-axis is '1-specificity'.

In [0]:
# Change the x to (1-spe) for the ROC curve
plt.step(x=1-np.array(lor_pred_spe), y=np.array(lor_pred_sen), where='post')
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.title('Receiver Operating Charcteristics Curve')

The area under the ROC curve (AUC of ROC) is a measure of model performance.

In summary, in terms of robustness:
$$ROC >  (sen, spe) > accuracy$$




In [0]:
# Print the session information for reproducibility purposes
import IPython
print(IPython.sys_info())



---

This is the end of the session.